# 밑바닥 딥러닝
## Chapter 05 Part 1
## 오차역전파법 


오차를 역(반대 방향)으로 전파하는 방법 (backward propagation of errors).

앞 장에서는 신경망의 가중치 매개변수의 기울기 (=가중치 매개변수에 대한 손실 함수의 기울기)를 수치 미분으로 계산했음

수치 미분은 단순하고 구현하기도 쉬우나 계산 시간이 오래걸린다는 게 단점.

# 5.1 계산 그래프
# 5.2. 연쇄 법칙
# 5.3. 역전파
# 5.4. 단순한 계층 구현


### 5.4.1. 곱셈 계층

모든 계층은 forward()와 backward()라는 공통의 메서드(인터페이스)를 갖도록 구현한다. 
forward()는 순전파, backward()는 역전파를 처리한다. 

먼저 곱셈 계층을 구현해보자. 곱셈 계층은 MulLayer라는 이름의 클래스로 다음과 같이 구현 가능하다. 

In [1]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/layer_naive.py 소스 참고
class MulLayer:
    def __init__(self): 
        #인스턴스 변수인 x와 y를 초기화한다. 
        #이 두 변수는 순전파 시의 입력 값을 유지하기 위해 사용한다.
        self.x = None
        self.y = None
    
    #forward에서는 x와 y를 인수로 받고 두 값을 곱해서 반환한다. 
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    #backward에서는 상류에서 넘어온 미분(dout)에 순전파 때의 값을 '서로 바꿔' 곱한 후 하류로 흘린다.'
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x 
        
        return dx, dy
    


MulLayer를 사용해서 앞에서 본 '사과 쇼핑'을 구현해보자. 
앞 절에서는 계산 그래프의 순전파와 역전파를 써서 [그림 5-16]과 같이 계산할 수 있다. 

In [3]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/buy_apple.py 소스 참고
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220


220.00000000000003


각 변수에 대한 미분은 backward()에서 구할 수 있다. 

In [6]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("사과 가격 대한 미분은",dapple, "사과 개수에 대한 미분은",dapple_num, "소비세에 대한 미분은",dtax) # 2.2 110 200

사과 가격 대한 미분은 2.2 사과 개수에 대한 미분은 110.00000000000001 소비세에 대한 미분은 200



backward()의 호출 순서는 forward() 때와 반대이다. 
backward()가 받는 인수는 '순전파의 출력에 대한 미분'임을 주의. 

가령 mul_apple_layer 라는 곱셈 계층은 순전파때는 apple_price를 출력하지만
역전파에서는 apple_price 의 미분 값인 ```dapple_price```를 인수로 받는다. 
마지막으로 이 코드를 실행한 결과는 [그림5-16]의 결과와 일치한다. 


### 5.4.2. 덧셈 계층

덧셈 노드인 덧셈 계층을 구현해보자. 

In [7]:
class AddLayer:
    def __init__(self): 
        pass #pass = 아무 실행하지 않음
    
    def forward(self, x, y): #입력받은 두 인수 x, y를 더해서 반환한다. 
        out = x + y 
        return out

    def backward(self, dout): #상류에서 내려온 미분(dout)을 그대로 하류로 흘린다. 
        dx = dout * 1
        dy = dout * 1
        return dx, dy

이상의 덧셈 계층과 곱셈 계층을 사용하여 사과 2개와 귤 3개를 사는 [그림 5-17]의 상황을 구현해보자

In [8]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch05/buy_apple.py 소스 참고
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파 : 필요한 계층을 만들어 순전파 메서드를 적절한 순서로 호출
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# 역전파 : 순전파와 반대 순서로 역전파 메서드를 호출하면 원하는 미분이 나옴
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price)) # 715
print("dApple:", dapple) # 2.2
print("dApple_num:", int(dapple_num)) # 110
print("dOrange:", dorange) # 3.3
print("dOrange_num:", int(dorange_num)) # 165
print("dTax:", dtax) # 650


price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650
